In [1]:
import pandas as pd
import numpy as np

# Load Data

## Load area mappings

In [307]:
area_df = pd.read_table("ap_area.txt")
#remove spaces in column headers
area_df.columns = area_df.columns.str.replace(' ','')

area_df


,area_code,area_name
0,0000,U.S. city average
1,0100,Northeast
2,0110,New England
3,0120,Middle Atlantic
4,0200,Midwest
...,...,...
69,S49C,"Riverside-San Bernardino-Ontario, CA"
70,S49D,Seattle-Tacoma-Bellevue WA
71,S49E,"San Diego-Carlsbad, CA"
72,S49F,Urban Hawaii


## create dates df

In [345]:
dates_df = pd.DataFrame(pd.date_range('1980-01-01','2022-08-01', freq='MS'), columns=['report_date'])

dates_df['year'] = dates_df['report_date'].dt.year
dates_df['month'] = dates_df['report_date'].dt.month
dates_df['day'] = dates_df['report_date'].dt.day

dates_df

,report_date,year,month,day
0,1980-01-01,1980,1,1
1,1980-02-01,1980,2,1
2,1980-03-01,1980,3,1
3,1980-04-01,1980,4,1
4,1980-05-01,1980,5,1
...,...,...,...,...
507,2022-04-01,2022,4,1
508,2022-05-01,2022,5,1
509,2022-06-01,2022,6,1
510,2022-07-01,2022,7,1


## Load food series values

In [346]:
# pull in data
food_df = pd.read_table("ap_data_3_Food.txt", low_memory = False)

# remove spaces in column headers
food_df.columns = food_df.columns.str.replace(' ','')

# remove white space from column values
food_df['series_id'] = food_df['series_id'].str.strip()

# get year month dat
food_df['year'] = food_df['year'].astype(int)
food_df['month'] = food_df.period.str[1:].astype(int)
food_df['day'] = 1

# convert to date
food_df["report_date"] = pd.to_datetime(food_df[['year', 'month', 'day']])

# create new price column
food_df['price'] = food_df['value'].str.strip().str.replace('-', 'nan').fillna(0).astype(float)

# get rid of these
food_df.drop(columns=['footnote_codes', 'period', 'value'], inplace=True)

# create full series, date mapping
food_full_dates_df = pd.merge(pd.DataFrame(food_df['series_id'].unique().tolist(), columns=['series_id']), dates_df, how='cross')


# rejoin so we have all months per series
food_df = pd.merge(food_full_dates_df, food_df, on=['series_id', 'report_date', 'year', 'month', 'day'], how='left')

# forward fill our prices so that we can avoid nulls on yoy growth / percent difference
food_df = food_df.sort_values(by=['series_id', 'report_date', 'year', 'month', 'day']).fillna(method="ffill")

In [347]:
# food_df[food_df.year.eq(2020) & food_df.series_id.eq('APU0000712311')]
food_df

,series_id,report_date,year,month,day,price
0,APU0000701111,1980-01-01,1980,1,1,0.203
1,APU0000701111,1980-02-01,1980,2,1,0.205
2,APU0000701111,1980-03-01,1980,3,1,0.211
3,APU0000701111,1980-04-01,1980,4,1,0.206
4,APU0000701111,1980-05-01,1980,5,1,0.207
...,...,...,...,...,...,...
344059,APU0400FS1101,2022-04-01,2022,4,1,4.529
344060,APU0400FS1101,2022-05-01,2022,5,1,4.529
344061,APU0400FS1101,2022-06-01,2022,6,1,4.590
344062,APU0400FS1101,2022-07-01,2022,7,1,4.616


## Load series mappings

In [348]:
series_df = pd.read_table("ap_series.txt")

# remove whitespace from column names
series_df.columns = series_df.columns.str.replace(' ','')

# remove whitespace from column values
series_strip_cols = ['series_id', 'series_title', 'area_code', 'item_code']
series_df[series_strip_cols] = series_df[series_strip_cols].apply(lambda x: x.str.strip())

# convert to int
series_df[['begin_year', 'end_year']] = series_df[['begin_year', 'end_year']].astype(int)
series_df['begin_month'] = series_df.begin_period.str[1:].astype(int)
series_df['end_month'] = series_df.end_period.str[1:].astype(int)

series_df.drop(columns=['footnote_codes', 'begin_period', 'end_period'], inplace=True)
    
series_df

,series_id,area_code,item_code,series_title,begin_year,end_year,begin_month,end_month
0,APU0000701111,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8
1,APU0000701311,0000,701311,"Rice, white, long grain, precooked (cost per p...",1980,1981,1,12
2,APU0000701312,0000,701312,"Rice, white, long grain, uncooked, per lb. (45...",1980,2022,1,8
3,APU0000701321,0000,701321,Spaghetti (cost per pound/453.6 grams) in U.S....,1980,1981,1,3
4,APU0000701322,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,2022,1,8
...,...,...,...,...,...,...,...,...
1477,APUS49G74713,S49G,74713,"Gasoline, leaded premium (cost per gallon/3.8 ...",1978,1981,1,4
1478,APUS49G74714,S49G,74714,"Gasoline, unleaded regular, per gallon/3.785 l...",1978,2022,1,8
1479,APUS49G74715,S49G,74715,"Gasoline, unleaded midgrade, per gallon/3.785 ...",2021,2022,6,8
1480,APUS49G74716,S49G,74716,"Gasoline, unleaded premium, per gallon/3.785 l...",1981,2022,9,8


## Load item mappings

In [349]:
item_df = pd.read_table("ap_item.txt")


# remove whitespace from column names
item_df.columns = item_df.columns.str.replace(' ','')

# remove whitespace from column values
item_strip_cols = ['item_name', 'item_code']
item_df[item_strip_cols] = item_df[item_strip_cols].apply(lambda x: x.str.strip())


item_df

,item_code,item_name
0,701111,"Flour, white, all purpose, per lb. (453.6 gm)"
1,701311,"Rice, white, long grain, precooked (cost per p..."
2,701312,"Rice, white, long grain, uncooked, per lb. (45..."
3,701321,Spaghetti (cost per pound/453.6 grams)
4,701322,"Spaghetti and macaroni, per lb. (453.6 gm)"
...,...,...
155,FJ4101,"Yogurt, per 8 oz. (226.8 gm)"
156,FL2101,"Lettuce, romaine, per lb. (453.6 gm)"
157,FN1101,"All soft drinks, per 2 liters (67.6 oz)"
158,FN1102,"All soft drinks, 12 pk, 12 oz., cans, per 12 o..."


## Merge datasets

In [350]:
df = food_df.merge(series_df, how = "left", on = "series_id") \
          .merge(item_df, how="left", on="item_code") \
          .merge(area_df, how = "left", on = "area_code")
df

,series_id,report_date,year,month,day,price,area_code,item_code,series_title,begin_year,end_year,begin_month,end_month,item_name,area_name
0,APU0000701111,1980-01-01,1980,1,1,0.203,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
1,APU0000701111,1980-02-01,1980,2,1,0.205,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
2,APU0000701111,1980-03-01,1980,3,1,0.211,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
3,APU0000701111,1980-04-01,1980,4,1,0.206,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
4,APU0000701111,1980-05-01,1980,5,1,0.207,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344059,APU0400FS1101,2022-04-01,2022,4,1,4.529,0400,FS1101,"Butter, stick, per lb. (453.6 gm) in West, ave...",2018,2022,4,8,"Butter, stick, per lb. (453.6 gm)",West
344060,APU0400FS1101,2022-05-01,2022,5,1,4.529,0400,FS1101,"Butter, stick, per lb. (453.6 gm) in West, ave...",2018,2022,4,8,"Butter, stick, per lb. (453.6 gm)",West
344061,APU0400FS1101,2022-06-01,2022,6,1,4.590,0400,FS1101,"Butter, stick, per lb. (453.6 gm) in West, ave...",2018,2022,4,8,"Butter, stick, per lb. (453.6 gm)",West
344062,APU0400FS1101,2022-07-01,2022,7,1,4.616,0400,FS1101,"Butter, stick, per lb. (453.6 gm) in West, ave...",2018,2022,4,8,"Butter, stick, per lb. (453.6 gm)",West


# Process Data

#### Filter merged dataset to include data from last 20 years where we have a full range of values (ie data from 2002 to 2022 M08)

In [351]:
##filter to only include data for last twenty years, where we have a fuller set of information for that series
df_filtered = df[df.begin_year.lt(2002) & 
                 df.end_year.eq(2022) &
                 df.end_month.eq(8) &
                 df.year.ge(2002)
                ]
df_filtered

,series_id,report_date,year,month,day,price,area_code,item_code,series_title,begin_year,end_year,begin_month,end_month,item_name,area_name
264,APU0000701111,2002-01-01,2002,1,1,0.319,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
265,APU0000701111,2002-02-01,2002,2,1,0.318,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
266,APU0000701111,2002-03-01,2002,3,1,0.310,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
267,APU0000701111,2002-04-01,2002,4,1,0.307,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
268,APU0000701111,2002-05-01,2002,5,1,0.323,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",1980,2022,1,8,"Flour, white, all purpose, per lb. (453.6 gm)",U.S. city average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340475,APU0400FD4101,2022-04-01,2022,4,1,3.859,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,2022,1,8,All Other Pork (Excluding Canned Ham and Lunch...,West
340476,APU0400FD4101,2022-05-01,2022,5,1,3.860,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,2022,1,8,All Other Pork (Excluding Canned Ham and Lunch...,West
340477,APU0400FD4101,2022-06-01,2022,6,1,4.037,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,2022,1,8,All Other Pork (Excluding Canned Ham and Lunch...,West
340478,APU0400FD4101,2022-07-01,2022,7,1,3.765,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,2022,1,8,All Other Pork (Excluding Canned Ham and Lunch...,West


#### Filter merged dataset to exclude items that don't exist for all geographies so we're looking at a consistent set of items

In [352]:
record_count_by_item_area = (pd.DataFrame({'count' : df_filtered.groupby( [ "area_name", "item_name"] ).size()})
                                         .reset_index()
                                         .pivot(index="item_name", columns="area_name", values="count")
                                         .dropna(axis=0)
                                         .reset_index()
                            )

items_for_analysis = record_count_by_item_area["item_name"].unique().tolist()
items_for_analysis

['All Ham (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)',
 'All Other Pork (Excluding Canned Ham and Luncheon Slices), per lb. (453.6 gm)',
 'All Pork Chops, per lb. (453.6 gm)',
 'All Uncooked Beef Roasts, per lb. (453.6 gm)',
 'All Uncooked Beef Steaks, per lb. (453.6 gm)',
 'All Uncooked Other Beef (Excluding Veal), per lb. (453.6 gm)',
 'All uncooked ground beef, per lb. (453.6 gm)',
 'Bacon, sliced, per lb. (453.6 gm)',
 'Bananas, per lb. (453.6 gm)',
 'Bread, white, pan, per lb. (453.6 gm)',
 'Cheddar cheese, natural, per lb. (453.6 gm)',
 'Chicken, fresh, whole, per lb. (453.6 gm)',
 'Chops, boneless, per lb. (453.6 gm)',
 'Ground beef, 100% beef, per lb. (453.6 gm)',
 'Ice cream, prepackaged, bulk, regular, per 1/2 gal. (1.9 lit)',
 'Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml)',
 'Potato chips, per 16 oz.',
 'Potatoes, white, per lb. (453.6 gm)',
 'Spaghetti and macaroni, per lb. (453.6 gm)',
 'Steak, sirloin, USDA Choice, boneless, 

## Clean up

In [353]:
# filter
final_df = df_filtered[df_filtered["item_name"].isin(items_for_analysis)].copy()

# # rename to fit pd.to_datetime reqs
# final_df.rename(columns={'value': 'price', 
#                          'period_num': 'month', 
#                          'day_num': 'day'}, inplace=True)

# # create date
# final_df["report_date"] = pd.to_datetime(final_df[['year', 'month', 'day']])

# clean up dtypes:
# final_df['price'] = final_df['price'].astype('double')
# final_df['month'] = final_df['month'].astype('int')


# sort and forward fill so that we only have null prices at the beginning 
# (sets to last month's price if null)
# final_df = final_df.sort_values(by=['area_name', 'item_name', 'year', 'month']).fillna(method="ffill")

## Create Categorical Variables

In [354]:
# parse item names for category key words
item_category_conditions = [
final_df['item_name'].str.lower().str.contains('ham|steak|beef|chicken|bacon|pork|boneless'),
final_df['item_name'].str.lower().str.contains('tomatoes|potatoes|bananas'),
final_df['item_name'].str.lower().str.contains('wine|malt'),
final_df['item_name'].str.lower().str.contains('ice cream|cheese'),
final_df['item_name'].str.lower().str.contains('bread|spaghetti'),
final_df['item_name'].str.lower().str.contains('chips')
]

# assign labels
item_category_labels = ['Meat', 'Produce', 'Alcohol', 'Dairy', 'Grains', 'Snacks']

# create the column
final_df['category'] = np.select(item_category_conditions, item_category_labels)

In [355]:
final_df[final_df.category.eq('Dairy')][['item_name', 'category']]
# final_df[final_df.category.eq('0')][['item_name', 'category']]

,item_name,category
33544,"Cheddar cheese, natural, per lb. (453.6 gm)",Dairy
33545,"Cheddar cheese, natural, per lb. (453.6 gm)",Dairy
33546,"Cheddar cheese, natural, per lb. (453.6 gm)",Dairy
33547,"Cheddar cheese, natural, per lb. (453.6 gm)",Dairy
33548,"Cheddar cheese, natural, per lb. (453.6 gm)",Dairy
...,...,...
306683,"Ice cream, prepackaged, bulk, regular, per 1/2...",Dairy
306684,"Ice cream, prepackaged, bulk, regular, per 1/2...",Dairy
306685,"Ice cream, prepackaged, bulk, regular, per 1/2...",Dairy
306686,"Ice cream, prepackaged, bulk, regular, per 1/2...",Dairy


## Calculate Values

### 12 month rolling average w/ YoY price change

In [356]:
final_df.dtypes

series_id               object
report_date     datetime64[ns]
year                     int64
month                    int64
day                      int64
price                  float64
area_code               object
item_code               object
series_title            object
begin_year               int64
end_year                 int64
begin_month              int64
end_month                int64
item_name               object
area_name               object
category                object
dtype: object

In [357]:
# get price for area, item at same time last year:
final_df['12mo_ma_price'] = final_df.sort_values(by=['year','month'], ascending=True) \
                                            .groupby(['area_name', 'item_name'])['price'].transform(lambda x: x.rolling(12, 1).mean())


# get price for area, item at same time last year:
final_df['ly_12mo_ma_price'] = final_df.sort_values(by=['month', 'year'], ascending=True) \
                                .groupby(['area_name', 'item_name'])['12mo_ma_price'].shift(1)



# calculate price change, absolute and percent change
final_df['yoy_12mo_ma_price_change'] = (final_df['12mo_ma_price'] - final_df['ly_12mo_ma_price'])
final_df['yoy_12mo_ma_price_percent_change'] =  round(100 * final_df['yoy_12mo_ma_price_change'] / final_df['ly_12mo_ma_price'], 2)





In [358]:
# get price for area, item at same time last year:
final_df['ly_price'] = final_df.sort_values(by=['month', 'year'], ascending=True) \
                                .groupby(['area_name', 'item_name'])['price'].shift(1)


# calculate price change, absolute and percent change
final_df['yoy_price_change'] = (final_df['price'] - final_df['ly_price'])
final_df['yoy_price_percent_change'] =  round(100 * final_df['yoy_price_change'] / final_df['ly_price'], 2)



### prove & show

In [359]:
# show that the above works  & final_df.month.eq(4)
final_df[final_df.area_name.eq('U.S. city average') & 
         final_df.item_name.str.contains('Tomatoes') &
         final_df.month.isin([2])] \
        [['series_id', 'area_name', 'item_name', 
          'month', 'year', '12mo_ma_price', 'ly_12mo_ma_price', 
          'yoy_price_change', 'yoy_price_percent_change']].style


,series_id,area_name,item_name,month,year,12mo_ma_price,ly_12mo_ma_price,yoy_price_change,yoy_price_percent_change
42249,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2002,1.374500,1.846083,-0.594000,-31.400000
42261,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2003,1.368417,1.374500,0.267000,20.570000
42273,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2004,1.484583,1.368417,-0.055000,-3.510000
42285,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2005,1.615167,1.484583,-0.082000,-5.430000
42297,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2006,1.693083,1.615167,0.482000,33.750000
42309,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2007,1.664417,1.693083,-0.266000,-13.930000
42321,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2008,1.688417,1.664417,0.091000,5.540000
42333,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2009,1.698250,1.688417,-0.179000,-10.320000
42345,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2010,1.654833,1.698250,0.209000,13.430000
42357,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2011,1.677417,1.654833,0.067000,3.800000


In [360]:
# show that the above works  & final_df.month.eq(4)
final_df[final_df.area_name.eq('U.S. city average') & 
         final_df.item_name.str.contains('Tomatoes') &
         final_df.year.eq(2012)] \
        [['series_id', 'area_name', 'item_name', 
          'month', 'year', 'price', 'ly_price', 
          'yoy_price_change', 'yoy_price_percent_change']].style


,series_id,area_name,item_name,month,year,price,ly_price,yoy_price_change,yoy_price_percent_change
42368,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",1,2012,1.542000,1.590000,-0.048000,-3.020000
42369,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",2,2012,1.433000,1.832000,-0.399000,-21.780000
42370,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",3,2012,1.452000,2.086000,-0.634000,-30.390000
42371,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",4,2012,1.388000,2.270000,-0.882000,-38.850000
42372,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",5,2012,1.336000,1.794000,-0.458000,-25.530000
42373,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",6,2012,1.492000,1.559000,-0.067000,-4.300000
42374,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",7,2012,1.506000,1.442000,0.064000,4.440000
42375,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",8,2012,1.440000,1.420000,0.020000,1.410000
42376,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",9,2012,1.384000,1.503000,-0.119000,-7.920000
42377,APU0000712311,U.S. city average,"Tomatoes, field grown, per lb. (453.6 gm)",10,2012,1.443000,1.465000,-0.022000,-1.500000


In [361]:
# show that the above works  & final_df.month.eq(4)
final_df[final_df.area_name.eq('U.S. city average') & 
         final_df.item_name.str.contains('Tomatoes') & 
         final_df.year.eq(2020)]

,series_id,report_date,year,month,day,price,area_code,item_code,series_title,begin_year,...,item_name,area_name,category,12mo_ma_price,ly_12mo_ma_price,yoy_12mo_ma_price_change,yoy_12mo_ma_price_percent_change,ly_price,yoy_price_change,yoy_price_percent_change
42464,APU0000712311,2020-01-01,2020,1,1,2.221,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.930000,1.907250,0.022750,1.19,2.227,-0.006,-0.27
42465,APU0000712311,2020-02-01,2020,2,1,2.197,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.940750,1.910750,0.030000,1.57,2.068,0.129,6.24
42466,APU0000712311,2020-03-01,2020,3,1,2.197,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.950583,1.928333,0.022250,1.15,2.079,0.118,5.68
42467,APU0000712311,2020-04-01,2020,4,1,2.197,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.971417,1.940417,0.031000,1.60,1.947,0.250,12.84
42468,APU0000712311,2020-05-01,2020,5,1,1.851,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.973750,1.952750,0.021000,1.08,1.823,0.028,1.54
42469,APU0000712311,2020-06-01,2020,6,1,1.922,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.982917,1.961583,0.021333,1.09,1.812,0.110,6.07
42470,APU0000712311,2020-07-01,2020,7,1,1.977,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.997750,1.954250,0.043500,2.23,1.799,0.178,9.89
42471,APU0000712311,2020-08-01,2020,8,1,1.874,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.998583,1.964833,0.033750,1.72,1.864,0.010,0.54
42472,APU0000712311,2020-09-01,2020,9,1,1.847,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,1.993250,1.971917,0.021333,1.08,1.911,-0.064,-3.35
42473,APU0000712311,2020-10-01,2020,10,1,1.911,0000,712311,"Tomatoes, field grown, per lb. (453.6 gm) in U...",1980,...,"Tomatoes, field grown, per lb. (453.6 gm)",U.S. city average,Produce,2.004250,1.961750,0.042500,2.17,1.779,0.132,7.42


# ***Write Data***

In [362]:
#write final data to csv
final_df.to_csv("output_cpi_data.csv")
final_df

,series_id,report_date,year,month,day,price,area_code,item_code,series_title,begin_year,...,item_name,area_name,category,12mo_ma_price,ly_12mo_ma_price,yoy_12mo_ma_price_change,yoy_12mo_ma_price_percent_change,ly_price,yoy_price_change,yoy_price_percent_change
2312,APU0000701322,2002-01-01,2002,1,1,0.912,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,...,"Spaghetti and macaroni, per lb. (453.6 gm)",U.S. city average,Grains,0.912000,NaN,NaN,NaN,NaN,NaN,NaN
2313,APU0000701322,2002-02-01,2002,2,1,0.902,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,...,"Spaghetti and macaroni, per lb. (453.6 gm)",U.S. city average,Grains,0.907000,1.162750,-0.255750,-22.00,1.116,-0.214,-19.18
2314,APU0000701322,2002-03-01,2002,3,1,0.917,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,...,"Spaghetti and macaroni, per lb. (453.6 gm)",U.S. city average,Grains,0.910333,1.148083,-0.237750,-20.71,1.148,-0.231,-20.12
2315,APU0000701322,2002-04-01,2002,4,1,0.927,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,...,"Spaghetti and macaroni, per lb. (453.6 gm)",U.S. city average,Grains,0.914500,1.134833,-0.220333,-19.42,1.182,-0.255,-21.57
2316,APU0000701322,2002-05-01,2002,5,1,0.901,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",1984,...,"Spaghetti and macaroni, per lb. (453.6 gm)",U.S. city average,Grains,0.911800,1.132000,-0.220200,-19.45,1.217,-0.316,-25.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340475,APU0400FD4101,2022-04-01,2022,4,1,3.859,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,...,All Other Pork (Excluding Canned Ham and Lunch...,West,Meat,3.749750,3.147500,0.602250,19.13,3.320,0.539,16.23
340476,APU0400FD4101,2022-05-01,2022,5,1,3.860,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,...,All Other Pork (Excluding Canned Ham and Lunch...,West,Meat,3.778833,3.200083,0.578750,18.09,3.511,0.349,9.94
340477,APU0400FD4101,2022-06-01,2022,6,1,4.037,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,...,All Other Pork (Excluding Canned Ham and Lunch...,West,Meat,3.802250,3.221750,0.580500,18.02,3.756,0.281,7.48
340478,APU0400FD4101,2022-07-01,2022,7,1,3.765,0400,FD4101,All Other Pork (Excluding Canned Ham and Lunch...,1998,...,All Other Pork (Excluding Canned Ham and Lunch...,West,Meat,3.798500,3.264167,0.534333,16.37,3.810,-0.045,-1.18
